link to original notebook: 
https://deepnote.com/workspace/my-space-d003-e2bb83f7-45c6-44c1-9ca3-2d2fdded6f46/project/Welcome-7a0c0182-8404-4a4a-b0b7-199d3e510618/notebook/Pivotalgorithm-3dc9594cbe624f9781f3fc7dcecc4ce3?utm_content=7a0c0182-8404-4a4a-b0b7-199d3e510618

this notebook implements 3 types of pivot algorithms, including:
1) the original pivot algorithm, 
2) telescoping method,
3) recursive method.
The last 2 methods share the same theory behind.
We also explore the GPU version of the algorithm.

# 1 Pivot Algorithm with Atmosphere Statistics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

class SAWSimulator:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        
    def generate_initial_saw(self, L):
        """Create straight initial SAW (non-equilibrium starting point)"""
        return np.array([[i,0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        """Perform one pivot attempt on a SAW"""
        L = len(walk) - 1
        k = np.random.randint(1, L)  # Random pivot point (can't be endpoint)
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        # Apply symmetry operation to subwalk
        subwalk = walk[k:] - walk[k]
        if g == 'rotate90':
            transformed = np.array([[-y,x] for [x,y] in subwalk])
        elif g == 'rotate180':
            transformed = np.array([[-x,-y] for [x,y] in subwalk])
        elif g == 'reflect_x':
            transformed = np.array([[x,-y] for [x,y] in subwalk])
        else:  # reflect_y
            transformed = np.array([[-x,y] for [x,y] in subwalk])
        
        # Check self-avoidance
        new_walk = np.concatenate([walk[:k], walk[k] + transformed])
        if len(np.unique(new_walk, axis=0)) == len(new_walk):
            return new_walk
        return walk
    
    def simulate_atmosphere(self, L, num_samples):
        """Estimate atmosphere statistics for walks of length L"""
        atm_counts = []
        walk = self.generate_initial_saw(L)
        
        # Thermalization (burn-in)
        start = time()
        for _ in range(10 * L):
            walk = self.pivot_move(walk)
        print(f"Thermalized {10 * L} pivots in {time()-start:.2f}s")
        
        # Production run
        for _ in range(num_samples):
            walk = self.pivot_move(walk)
            # Count possible extensions
            last_point = walk[-1]
            count = 0
            for d in self.directions:
                new_point = last_point + d
                if not any(np.all(new_point == walk, axis=1)):
                    count += 1
            atm_counts.append(count)
            
        return np.mean(atm_counts), np.std(atm_counts) / np.sqrt(len(atm_counts))
    
    def estimate_mu(self, num_samples=10000):
        """Estimate μ using atmosphere statistics"""
        lengths = np.arange(4, self.L_max+1, 2)  # Even lengths to avoid parity effects
        atm_means = []
        atm_stds = []
        
        print("Running SAW simulations for μ estimation...")
        for L in tqdm(lengths):
            mean_atm, std_atm = self.simulate_atmosphere(L, num_samples)
            atm_means.append(mean_atm)
            atm_stds.append(std_atm)
            print(f"L={L}: ⟨a⟩ = {mean_atm:.3f} ± {std_atm:.3f}")
        
        # Fit to extract μ and γ
        def model(n, mu, gamma, c):
            return mu * (1 + (gamma-1)/n + c/n**2)
        
        popt, pcov = curve_fit(model, lengths, atm_means)
        mu, gamma = popt[0], popt[1]
        
        # Plot results
        plt.figure(figsize=(10,6))
        plt.scatter(1/lengths, atm_means, label='Simulation Data')
        x_fit = np.linspace(0, 1/lengths[0], 100)
        plt.plot(x_fit, model(1/x_fit, *popt), 'r-', 
                label=f'Fit: μ={mu:.6f}, γ={gamma:.6f}')
        plt.xlabel('1/L')
        plt.ylabel('<a>')
        plt.title('Atmosphere Statistics for 2D SAWs')
        plt.legend()
        plt.grid(True)
        plt.savefig('saw_mu_estimation.png', dpi=300)
        plt.close()
        
        return mu, gamma

# Benchmarking
if __name__ == "__main__":
    simulator = SAWSimulator(L_max=100)
    
    start_time = time()
    mu, gamma = simulator.estimate_mu(num_samples=50)
    cpu_time = time() - start_time
    
    print(f"CPU Version Results:")
    print(f"Estimated μ = {mu:.6f}")
    print(f"Estimated γ = {gamma:.6f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

Running SAW simulations for μ estimation...
 12%|█▏        | 6/49 [00:00<00:00, 51.17it/s]Thermalized 40 pivots in 0.01s
L=4: ⟨a⟩ = 2.720 ± 0.063
Thermalized 60 pivots in 0.01s
L=6: ⟨a⟩ = 2.860 ± 0.049
Thermalized 80 pivots in 0.01s
L=8: ⟨a⟩ = 2.440 ± 0.070
Thermalized 100 pivots in 0.01s
L=10: ⟨a⟩ = 2.200 ± 0.106
Thermalized 120 pivots in 0.02s
L=12: ⟨a⟩ = 2.740 ± 0.062
Thermalized 140 pivots in 0.01s
L=14: ⟨a⟩ = 2.660 ± 0.067
Thermalized 160 pivots in 0.03s
L=16: ⟨a⟩ = 2.860 ± 0.049
Thermalized 180 pivots in 0.03s
L=18: ⟨a⟩ = 3.000 ± 0.000
Thermalized 200 pivots in 0.03s
L=20: ⟨a⟩ = 2.260 ± 0.068
Thermalized 220 pivots in 0.04s
L=22: ⟨a⟩ = 2.840 ± 0.052
Thermalized 240 pivots in 0.03s
L=24: ⟨a⟩ = 2.900 ± 0.042
 24%|██▍       | 12/49 [00:00<00:01, 29.85it/s]Thermalized 260 pivots in 0.04s
L=26: ⟨a⟩ = 2.260 ± 0.123
Thermalized 280 pivots in 0.05s
L=28: ⟨a⟩ = 2.860 ± 0.049
Thermalized 300 pivots in 0.05s
L=30: ⟨a⟩ = 2.760 ± 0.060
Thermalized 320 pivots in 0.05s
L=32: ⟨a⟩ = 1.100 ± 0.110

## 2) Implement GPU version pivot algorithm with atmosphere statistics

### Note that this first implementation uses 2-layer for-loop to perform collision check, which leads to computation complexity of $O(ThermilisationStep \times L^2) \approx O(L^3)$.

### this implementation needs improvement

In [2]:
# implement GPU version pivot algorithm with atmosphere statistics
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
from time import time
from pycuda import gpuarray
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

class SAWSimulatorGPU:
    def __init__(self, L_max=100):
        """Initialize GPU-accelerated SAW simulator with separated C/C++ compilation"""
        self.L_max = L_max
        self.block_size = 128
        self.grid_size = 2048
        self.total_walks = self.block_size * self.grid_size
        
        # 分步编译：先编译C++部分，再链接
        cuda_code = """
        #include <curand_kernel.h>
        
        __device__ void apply_symmetry(int* walk, int L, int k, int sym_op) {
            int dx, dy;
            for (int i = k+1; i <= L; i++) {
                dx = walk[2*i] - walk[2*k];
                dy = walk[2*i+1] - walk[2*k+1];
                
                if (sym_op == 0) { // rotate90
                    walk[2*i] = walk[2*k] - dy;
                    walk[2*i+1] = walk[2*k+1] + dx;
                } else if (sym_op == 1) { // rotate180
                    walk[2*i] = walk[2*k] - dx;
                    walk[2*i+1] = walk[2*k+1] - dy;
                } else if (sym_op == 2) { // reflect_x
                    walk[2*i] = walk[2*k] + dx;
                    walk[2*i+1] = walk[2*k+1] - dy;
                } else { // reflect_y
                    walk[2*i] = walk[2*k] - dx;
                    walk[2*i+1] = walk[2*k+1] + dy;
                }
            }
        }
        
        __device__ int check_self_avoiding(int* walk, int L) {
            for (int i = 0; i < L; i++) {
                for (int j = i+1; j <= L; j++) {
                    if (walk[2*i] == walk[2*j] && walk[2*i+1] == walk[2*j+1]) {
                        return 0;
                    }
                }
            }
            return 1;
        }
        
        extern "C" {
            __global__ void simulate_atmospheres(int* all_walks, int L, int min_success, float* results) {
                int idx = blockIdx.x * blockDim.x + threadIdx.x;
                int* walk = &all_walks[idx * 2 * (L+1)];
                
                curandState state;
                curand_init(clock64(), idx, 0, &state);
                
                int success = 0;
                while (success < min_success) {
                    int k = 1 + (int)((L-1) * curand_uniform(&state));
                    int sym_op = (int)(4 * curand_uniform(&state));
                    
                    int old_pos[2] = {walk[2*k], walk[2*k+1]};
                    apply_symmetry(walk, L, k, sym_op);
                    
                    if (check_self_avoiding(walk, L)) {
                        success++;
                    } else {
                        for (int i = k+1; i <= L; i++) {
                            walk[2*i] = old_pos[0] + (walk[2*i] - walk[2*k]);
                            walk[2*i+1] = old_pos[1] + (walk[2*i+1] - walk[2*k+1]);
                        }
                    }
                }
                
                int last_x = walk[2*L], last_y = walk[2*L+1];
                int directions[4][2] = {{1,0}, {-1,0}, {0,1}, {0,-1}};
                int count = 0;
                
                for (int d = 0; d < 4; d++) {
                    int collision = 0;
                    int new_x = last_x + directions[d][0];
                    int new_y = last_y + directions[d][1];
                    
                    for (int i = max(0, L-10); i <= L; i++) {
                        if (walk[2*i] == new_x && walk[2*i+1] == new_y) {
                            collision = 1;
                            break;
                        }
                    }
                    if (!collision) {
                        for (int i = 0; i < L-10; i++) {
                            if (walk[2*i] == new_x && walk[2*i+1] == new_y) {
                                collision = 1;
                                break;
                            }
                        }
                    }
                    count += (1 - collision);
                }
                results[idx] = (float)count;
            }
        }
        """
        
        # 编译选项（Kaggle专用）
        self.mod = SourceModule(
            cuda_code,
            options=[
                '-I/usr/local/cuda/include',
                '-L/usr/local/cuda/lib64',
                '-lcurand',
                '--expt-relaxed-constexpr',
                '--default-stream', 'per-thread'
            ],
            no_extern_c=True  
        )
        
        self.simulate_atmospheres = self.mod.get_function("simulate_atmospheres")

    def run_simulation(self, lengths=None, num_samples=10000):
        """主运行函数"""
        if lengths is None:
            lengths = np.arange(4, min(self.L_max, 100)+1, 2)
        
        atm_means = []
        atm_stds = []
        
        for L in lengths:
            # 初始化walks（全部从直线开始）
            walks = np.zeros((self.total_walks, L+1, 2), dtype=np.int32)
            walks[:,:,0] = np.arange(L+1)
            
            # 传输到GPU
            walks_gpu = gpuarray.to_gpu(walks.reshape(-1))
            results_gpu = gpuarray.zeros(self.total_walks, dtype=np.float32)
            
            # 执行核函数
            min_success = max(10 * L, 1000)
            self.simulate_atmospheres(
                walks_gpu, np.int32(L), np.int32(min_success), results_gpu,
                block=(self.block_size,1,1), grid=(self.grid_size,1))
            
            # 获取结果
            results = results_gpu.get()
            atm_means.append(np.mean(results))
            atm_stds.append(np.std(results) / np.sqrt(len(results)))
            
            print(f"L={L}: ⟨a⟩={atm_means[-1]:.3f}±{atm_stds[-1]:.3f}")
        
        return lengths, np.array(atm_means), np.array(atm_stds)

# Kaggle环境测试
if __name__ == "__main__":
    try:
        print("Initializing GPU...")
        simulator = SAWSimulatorGPU(L_max=100)
        
        print("Running simulation...")
        lengths, means, stds = simulator.run_simulation(
            lengths=np.arange(4, 51, 2),
            num_samples=1000
        )
        
        # 拟合曲线
        def model(L, mu, gamma, A):
            return mu * (1 + (gamma-1)/L + A/(L**2))
        
        popt, _ = curve_fit(model, lengths, means, sigma=stds)
        print(f"\nFitted μ = {popt[0]:.5f} (theory: 2.638)")
        print(f"Fitted γ = {popt[1]:.5f} (theory: 1.343)")
        
        # 绘图
        plt.errorbar(lengths, means, yerr=stds, fmt='o', label='Simulation')
        L_fit = np.linspace(min(lengths), max(lengths), 100)
        plt.plot(L_fit, model(L_fit, *popt), 'r-', label='Fit')
        plt.xlabel('Walk Length'); plt.ylabel('⟨a⟩')
        plt.legend(); plt.grid()
        plt.savefig('saw_results.png')
        
    except Exception as e:
        print(f"Error: {e}")
        print("Falling back to CPU...")
        # 此处添加CPU后备代码

## 3) Compare the computation time between CPU and GPU

In [3]:
# compare the computation time between CPU and GPU

# 2 Pivot Algorithm with Telescoping Method

## 1) Telecoping method with simple estimation method (test version)

In [4]:
# test for telescoping method
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

class SAWSimulatorCPU:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice with telescoping method"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        self.mu_estimate = 2.6381585  # Benchmark value from literature
        
    def generate_initial_saw(self, L):
        """Create straight initial SAW (non-equilibrium starting point)"""
        return np.array([[i,0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        """Perform one pivot attempt on a SAW"""
        L = len(walk) - 1
        k = np.random.randint(1, L)  # Random pivot point (can't be endpoint)
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        # Apply symmetry operation to subwalk
        subwalk = walk[k:] - walk[k]
        if g == 'rotate90':
            transformed = np.array([[-y,x] for [x,y] in subwalk])
        elif g == 'rotate180':
            transformed = np.array([[-x,-y] for [x,y] in subwalk])
        elif g == 'reflect_x':
            transformed = np.array([[x,-y] for [x,y] in subwalk])
        else:  # reflect_y
            transformed = np.array([[-x,y] for [x,y] in subwalk])
        
        # Check self-avoidance
        new_walk = np.concatenate([walk[:k], walk[k] + transformed])
        if len(np.unique(new_walk, axis=0)) == len(new_walk):
            return new_walk
        return walk
    
    def telescoping_estimate(self, N1, N2, num_samples):
        """Estimate B(N1,N2) = c_{N1+N2+1} / (c_N1 * c_N2) using telescoping method"""
        print(f"Telescoping(Concatenating) N1={N1}, N2={N2}")
        B_values = []
        
        for _ in range(num_samples):
            # Generate two independent SAWs
            walk1 = self.generate_initial_saw(N1)
            walk2 = self.generate_initial_saw(N2)
            
            # Thermalize both walks
            for _ in range(10*N1):
                walk1 = self.pivot_move(walk1)
            for _ in range(10*N2):
                walk2 = self.pivot_move(walk2)
            
            # Check concatenation
            concatenated = np.concatenate([walk1, walk2[1:] + walk1[-1] - walk2[0] + [1,0]])
            if len(np.unique(concatenated, axis=0)) == len(concatenated):
                B_values.append(1)
            else:
                B_values.append(0)
    
        return np.mean(B_values)
    
    def estimate_mu_telescoping(self, base_N=36, num_samples=10000):
        """Estimate μ using telescoping method with base_N as starting point"""
        # Create sequence of N values following the telescoping pattern
        N_sequence = [base_N]
        while N_sequence[-1] * 2 + 1 <= self.L_max:
            N_sequence.append(N_sequence[-1] * 2 + 1)
        
        print(f"Using telescoping sequence: {N_sequence}")
        
        # Estimate B values for each consecutive pair
        B_values = []
        for i in tqdm(range(len(N_sequence)-1), desc="Telescoping stages"):
            B = self.telescoping_estimate(N_sequence[i], N_sequence[i], num_samples)
            B_values.append(B)
            print(f"B({N_sequence[i]},{N_sequence[i]}) = {B:.6f}")
        
        # Calculate μ estimates at each stage
        mu_estimates = []
        for i in range(len(B_values)):
            # Simplified estimation - in practice would use exact c_N for base_N
            mu_est = (B_values[i] * 4) ** (1/(2*N_sequence[i]+1))  # 4 is coordination number
            mu_estimates.append(mu_est)
        
        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(N_sequence[1:], mu_estimates, 'b-', label='μ estimate')
        plt.axhline(y=self.mu_estimate, color='r', linestyle='--', 
                   label=f'Benchmark: {self.mu_estimate:.7f}')
        plt.xlabel('Walk Length (N)')
        plt.ylabel('μ estimate')
        plt.title(f'Telescoping Method (CPU)\nMax N={self.L_max}, Samples={num_samples}\nFinal μ={mu_estimates[-1]:.7f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('saw_telescoping_cpu.png', dpi=300)
        plt.show()
        
        return mu_estimates[-1]

# Main execution
if __name__ == "__main__":
    simulator = SAWSimulatorCPU(L_max=1000)
    
    start_time = time()
    mu_final = simulator.estimate_mu_telescoping(base_N=36, num_samples=1000)
    cpu_time = time() - start_time
    
    print(f"\nCPU Telescoping Version Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

## 2) Telescoping method using curve fitting for a more precise estimation (test version)

In [10]:
# test for telescoping method
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

class SAWSimulatorCPU:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice with telescoping method"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        self.mu_estimate = 2.6381585  # Benchmark value from literature
        
    def generate_initial_saw(self, L):
        """Create straight initial SAW (non-equilibrium starting point)"""
        return np.array([[i,0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        """Perform one pivot attempt on a SAW"""
        L = len(walk) - 1
        k = np.random.randint(1, L)  # Random pivot point (can't be endpoint)
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        # Apply symmetry operation to subwalk
        subwalk = walk[k:] - walk[k]
        if g == 'rotate90':
            transformed = np.array([[-y,x] for [x,y] in subwalk])
        elif g == 'rotate180':
            transformed = np.array([[-x,-y] for [x,y] in subwalk])
        elif g == 'reflect_x':
            transformed = np.array([[x,-y] for [x,y] in subwalk])
        else:  # reflect_y
            transformed = np.array([[-x,y] for [x,y] in subwalk])
        
        # Check self-avoidance
        new_walk = np.concatenate([walk[:k], walk[k] + transformed])
        if len(np.unique(new_walk, axis=0)) == len(new_walk):
            return new_walk
        return walk
    
    def telescoping_estimate(self, N1, N2, num_samples):
        """Estimate B(N1,N2) = c_{N1+N2+1} / (c_N1 * c_N2) using telescoping method"""
        print(f"Telescoping(Concatenating) N1={N1}, N2={N2}")
        B_values = []
        
        for _ in range(num_samples):
            # Generate two independent SAWs
            walk1 = self.generate_initial_saw(N1)
            walk2 = self.generate_initial_saw(N2)
            
            # Thermalize both walks
            for _ in range(10*N1):
                walk1 = self.pivot_move(walk1)
            for _ in range(10*N2):
                walk2 = self.pivot_move(walk2)
            
            # Check concatenation
            concatenated = np.concatenate([walk1, walk2[1:] + walk1[-1] - walk2[0] + [1,0]])
            if len(np.unique(concatenated, axis=0)) == len(concatenated):
                B_values.append(1)
            else:
                B_values.append(0)
    
        return np.mean(B_values)

    def estimate_mu_telescoping(self, base_N=36, num_samples=10000):
        """Estimate μ using telescoping method with base_N as starting point"""
        # Create sequence of N values following the telescoping pattern
        N_sequence = [base_N]
        while N_sequence[-1] * 2 + 1 <= self.L_max:
            N_sequence.append(N_sequence[-1] * 2 + 1)
        
        print(f"Using telescoping sequence: {N_sequence}")
        
        # Estimate B values for each consecutive pair and compute c_N
        B_values = []
        c_N = {base_N: 1.0}  # We'll work with relative counts, normalized to c_base_N = 1
        
        for i in tqdm(range(len(N_sequence)-1), desc="Telescoping stages"):
            N1 = N_sequence[i]
            N2 = N_sequence[i]
            N_total = 2*N1 + 1
            
            B = self.telescoping_estimate(N1, N2, num_samples)
            B_values.append(B)
            
            # Compute c_N using the telescoping relation: c_{2N+1} = B(N,N) * c_N^2
            c_N[N_total] = B * (c_N[N1] ** 2)
            
            print(f"B({N1},{N2}) = {B:.6f}, c_{N_total} = {c_N[N_total]:.6e}")
        
        # Prepare data for fitting equation (2.13)
        N_list = sorted(c_N.keys())
        log_mu_N = [np.log(c_N[N]) / N for N in N_list]
        
        # Define fitting function based on equation (2.13)
        def fitting_func(N, log_mu, A, y):
            return log_mu + (y-1)*np.log(N)/N + A/N
        
        # Perform the fit
        popt, pcov = curve_fit(fitting_func, N_list, log_mu_N, 
                            p0=[np.log(self.mu_estimate), 1.0, 1.0])
        
        log_mu_est, A_est, y_est = popt
        mu_est = np.exp(log_mu_est)
        
        # Calculate μ_N = c_N^(1/N) for plotting
        mu_N = [np.exp(ln_mu) for ln_mu in log_mu_N]
        
        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(N_list, mu_N, 'bo-', label='μ_N = c_N^(1/N)')
        plt.axhline(y=mu_est, color='g', linestyle='--', 
                label=f'Fitted μ: {mu_est:.7f}')
        plt.axhline(y=self.mu_estimate, color='r', linestyle='--', 
                label=f'Benchmark: {self.mu_estimate:.7f}')
        
        # Plot fitted curve
        fit_x = np.linspace(min(N_list), max(N_list), 100)
        fit_y = np.exp(fitting_func(fit_x, *popt))
        plt.plot(fit_x, fit_y, 'm--', label='Fitted curve')
        
        plt.xlabel('Walk Length (N)')
        plt.ylabel('μ estimate')
        plt.title(f'Telescoping Method (CPU)\nMax N={self.L_max}, Samples={num_samples}\n'
                f'Final μ={mu_est:.7f}, y={y_est:.3f}, A={A_est:.3f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('saw_telescoping_cpu.png', dpi=300)
        plt.show()
        
        return mu_est

# Main execution
if __name__ == "__main__":
    simulator = SAWSimulatorCPU(L_max=1000)
    
    start_time = time()
    mu_final = simulator.estimate_mu_telescoping(base_N=36, num_samples=1000)
    cpu_time = time() - start_time
    
    print(f"\nCPU Telescoping Version Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

## 3) summary and reflection on telescoping
telescoping method 仍是一个选项，不过我觉得需要在现有基础上优化，
1）比如现将 C_base 的长度降低，并且使用已知的确定值；
2）降低长度间隔；
3）当长度过长时，对walk 进行初始化会花费很长的时间，所以我希望在 burn in 阶段看能否进行一些优化（比如是否有比依次进行 pivot move 效率更高的方法）；
4）同时我记得要保证样本的独立性，需要进行L步pivot moves, 所以我觉得这里应该也要保证样本独立性，提升统计效率，采取 L 步获取新的 sample walk  
5)两个以及两个 walk 在 concatenation 时能否进行算法优化提升判断B值的速度;
6) 或整个算法采用论文提到的 SAW tree 数据结构
7) when doing pivot moves, choose some specific points instead of randomly choosing from L points

In [1]:
# Improved telescoping fit for small L_max

## 4) Implement SAW Tree

# 3 simple recursive method
telescoping method 每次长度都是 2N+1 的变化，间隔太大; recursive method将 N 的间隔调小。
前28个长度使用默认值: 1, 4, 12, 36, 100, 284, 780, 2172, 5916, 16268, 44100, 120292, 324932, 881500, 2374444, 6416596, 17245332, 46466676, 124658732, 335116620, 897697164, 2408806028, 6444560484, 17266613812, 46146397316, 123481354908, 329712786220, 881317491628

### the most important equation behind all the recursive method:
$$B(C_{N1}, C_{N2}) = \frac{C_{N1+N2}}{C_{N1} \cdot C_{N2}}$$

### the precise estimation version

In [3]:
# Simple recursive method
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from collections import defaultdict

class SAWSimulator:
    def __init__(self, L_max=1000):
        self.L_max = L_max
        # 已知的SAW计数（前28项）
        self.known_counts = {
            1:4, 2:12, 3:36, 4:100, 5:284, 6:780, 7:2172,
            8: 5916, 9:16268, 10:44100, 11:120292, 12:324932,
            13:881500, 14:2374444, 15:6416596,
            16:17245332, 17:46466676, 18:124658732,
            19:335116620, 20:897697164,
            21:2408806028, 22:6444560484,
            23:17266613812, 24:46146397316,
            25:123481354908, 26:329712786220,
            27:881317491628
        }
        self.directions = np.array([[1,0],[-1,0],[0,1],[0,-1]])  # 2D四方格
        
    def generate_initial_saw(self, L):
        """生成直线型初始SAW"""
        return np.array([[i,0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        """带拒绝采样的pivot移动"""
        L = len(walk) - 1
        k = np.random.randint(1, L)  # 随机枢轴点
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        subwalk = walk[k:] - walk[k]
        if g == 'rotate90':
            transformed = np.array([[-y,x] for [x,y] in subwalk])
        elif g == 'rotate180':
            transformed = np.array([[-x,-y] for [x,y] in subwalk])
        elif g == 'reflect_x':
            transformed = np.array([[x,-y] for [x,y] in subwalk])
        else:
            transformed = np.array([[-x,y] for [x,y] in subwalk])
            
        new_walk = np.concatenate([walk[:k], walk[k] + transformed])
        if len(np.unique(new_walk, axis=0)) == len(new_walk):
            return new_walk
        return walk

    def thermalize(self, walk, L):
        """优化的热化过程"""
        for _ in range(10 * L):
            walk = self.pivot_move(walk)
        return walk

    def recursive_estimate(self, L_target, num_samples=10000):
        """递归估计c_L"""
        if L_target in self.known_counts:
            return self.known_counts[L_target]
            
        # 寻找最大的已知L1 < L_target
        L1 = max(k for k in self.known_counts if k <= L_target//2)
        L2 = L_target - L1
        
        # 递归计算
        if L2 not in self.known_counts:
            c_L2 = self.recursive_estimate(L2, num_samples)
        else:
            c_L2 = self.known_counts[L2]
            
        # 估计B(L1,L2)
        B_est = self.estimate_B(L1, L2, num_samples)
        c_L = B_est * self.known_counts[L1] * c_L2
        
        # 缓存结果
        self.known_counts[L_target] = c_L
        return c_L

    def estimate_B(self, L1, L2, num_samples):
        """估计连接概率B(L1,L2)"""
        B_values = []
        
        for _ in tqdm(range(num_samples), desc=f"Estimating B({L1},{L2})"):
            # 生成并热化两个SAW
            walk1 = self.thermalize(self.generate_initial_saw(L1), L1)
            walk2 = self.thermalize(self.generate_initial_saw(L2), L2)
            
            # 检查连接
            concatenated = np.concatenate([
                walk1, 
                walk2[1:] + walk1[-1] - walk2[0] + [1,0]
            ])
            if len(np.unique(concatenated, axis=0)) == len(concatenated):
                B_values.append(1)
            else:
                B_values.append(0)
                
        return np.mean(B_values)

    def estimate_mu(self, num_samples=10000):
        """估计μ值"""
        L_sequence = sorted(self.known_counts.keys())
        L_sequence += [L for L in range(L_sequence[-1]+1, self.L_max+1) 
                      if L not in self.known_counts]
        
        for L in L_sequence:
            if L not in self.known_counts:
                self.recursive_estimate(L, num_samples)
        
        # 拟合μ值
        L_list = sorted(self.known_counts.keys())
        log_c = [np.log(self.known_counts[L]) for L in L_list]
        coeffs = np.polyfit(L_list, log_c, 1)
        mu_est = np.exp(coeffs[0])
        
        return mu_est
    
# Main execution
if __name__ == "__main__":
    simulator = SAWSimulator(L_max=100)
    
    start_time = time()
    mu_final = simulator.estimate_mu(num_samples=100)
    cpu_time = time() - start_time
    
    print(f"\nCPU Recursive Method Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")

Estimating B(50,50): 100%|██████████| 100/100 [00:28<00:00,  3.54it/s]
CPU Recursive Method Results:
Final μ estimate = 2.6815787
Computation Time = 1546.70 seconds



### Try using larger and more sparse L values to see if the results are more accurate.

# 4 Full configuration simulation
### using improved recursive method, precise estimation, SAW tree

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

class SAWSimulatorCPU:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice with telescoping method"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        self.mu_estimate = 2.6381585  # Benchmark value from literature
        
    def generate_initial_saw(self, L):
        """Create straight initial SAW (non-equilibrium starting point)"""
        return np.array([[i,0] for i in range(L+1)])
    
    def pivot_move(self, walk):
        """Perform one pivot attempt on a SAW"""
        L = len(walk) - 1
        k = np.random.randint(1, L)  # Random pivot point (can't be endpoint)
        g = np.random.choice(['rotate90', 'rotate180', 'reflect_x', 'reflect_y'])
        
        # Apply symmetry operation to subwalk
        subwalk = walk[k:] - walk[k]
        if g == 'rotate90':
            transformed = np.array([[-y,x] for [x,y] in subwalk])
        elif g == 'rotate180':
            transformed = np.array([[-x,-y] for [x,y] in subwalk])
        elif g == 'reflect_x':
            transformed = np.array([[x,-y] for [x,y] in subwalk])
        else:  # reflect_y
            transformed = np.array([[-x,y] for [x,y] in subwalk])
        
        # Check self-avoidance
        new_walk = np.concatenate([walk[:k], walk[k] + transformed])
        if len(np.unique(new_walk, axis=0)) == len(new_walk):
            return new_walk
        return walk
    
    def telescoping_estimate(self, N1, N2, num_samples):
        """Estimate B(N1,N2) = c_{N1+N2+1} / (c_N1 * c_N2) using telescoping method"""
        B_values = []
        
        # Progress bar for the simulation
        pbar = tqdm(total=num_samples, desc=f"Processing N1={N1}, N2={N2}")
        
        for _ in range(num_samples):
            # Generate two independent SAWs
            walk1 = self.generate_initial_saw(N1)
            walk2 = self.generate_initial_saw(N2)
            
            # Thermalize both walks
            for _ in range(10*N1):
                walk1 = self.pivot_move(walk1)
            for _ in range(10*N2):
                walk2 = self.pivot_move(walk2)
            
            # Check concatenation
            concatenated = np.concatenate([walk1, walk2[1:] + walk1[-1] - walk2[0] + [1,0]])
            if len(np.unique(concatenated, axis=0)) == len(concatenated):
                B_values.append(1)
            else:
                B_values.append(0)
            
            pbar.update(1)
        
        pbar.close()
        return np.mean(B_values)
    
    def estimate_mu_telescoping(self, base_N=36, num_samples=100000):
        """Estimate μ using telescoping method with base_N as starting point"""
        # Create sequence of N values following the telescoping pattern
        N_sequence = [base_N]
        while N_sequence[-1] * 2 + 1 <= self.L_max:
            N_sequence.append(N_sequence[-1] * 2 + 1)
        
        print(f"Using telescoping sequence: {N_sequence}")
        
        # Estimate B values for each consecutive pair
        B_values = []
        for i in tqdm(range(len(N_sequence)-1), desc="Telescoping stages"):
            B = self.telescoping_estimate(N_sequence[i], N_sequence[i], num_samples)
            B_values.append(B)
            print(f"B({N_sequence[i]},{N_sequence[i]}) = {B:.6f}")
        
        # Calculate μ estimates at each stage
        mu_estimates = []
        for i in range(len(B_values)):
            # Simplified estimation - in practice would use exact c_N for base_N
            mu_est = (B_values[i] * 4) ** (1/(2*N_sequence[i]+1))  # 4 is coordination number
            mu_estimates.append(mu_est)
        
        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(N_sequence[1:], mu_estimates, 'b-', label='μ estimate')
        plt.axhline(y=self.mu_estimate, color='r', linestyle='--', 
                   label=f'Benchmark: {self.mu_estimate:.7f}')
        plt.xlabel('Walk Length (N)')
        plt.ylabel('μ estimate')
        plt.title(f'Telescoping Method (CPU)\nMax N={self.L_max}, Samples={num_samples}\nFinal μ={mu_estimates[-1]:.7f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('saw_telescoping_cpu.png', dpi=300)
        plt.show()
        
        return mu_estimates[-1]

# Main execution
if __name__ == "__main__":
    simulator = SAWSimulatorCPU(L_max=1000)
    
    start_time = time()
    mu_final = simulator.estimate_mu_telescoping(base_N=36, num_samples=100000)
    cpu_time = time() - start_time
    
    print(f"\nCPU Telescoping Version Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {cpu_time:.2f} seconds")